##### Source:
imblearn Documentation
- https://github.com/scikit-learn-contrib/imbalanced-learn 
- http://scikit-learn.org/stable/auto_examples/model_selection/plot_precision_recall.html

In [1]:
import zipfile
import pandas as pd
import numpy as np
import glob
from scipy import sparse
from sklearn.model_selection import train_test_split
from collections import Counter
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import KFold
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
molecule_descriptor_files = []
for file in glob.glob('./Dataset/PubChem_Molecule_Descriptor/PubChem/AID_2289_descriptors_part[0-9]*.csv'):
    molecule_descriptor_files.append(file)

### Drug Bank

In [3]:
drug_link = pd.read_csv('./Dataset/drug_links.csv',dtype={'PubChem Compound ID':'str'})

In [4]:
drug_link = drug_link.dropna(subset=['PubChem Compound ID'])

In [5]:
drug_map = dict(zip(drug_link['PubChem Compound ID'],drug_link['Name']))

### Prepare training data

#### Use counter-screen (AID: 588342) to filter out false positive

Counter-screen: A screen performed in parallel with or after the primary screen. The assay used in the counter-screen is developed to identify compounds that have the potential to interfere with the assay used in the primary screen (the primary assay). Counter-screens can also be used to eliminate compounds that possess undesirable properties, for example, a counter-screen for cytotoxicity.

In [6]:
fp = pd.read_csv('./Dataset/AID_588342_descriptors_part1.csv',dtype={'Row':'str'})

In [7]:
fp_id = fp.iloc[:,0]
fp_id = fp_id[pd.notna(fp_id)]

In [8]:
# only first file contains active molecule, filter out false positive first
molecule_matrix = pd.read_csv(molecule_descriptor_files[0],dtype={'Row':'str'})
features = molecule_matrix.columns[1:180] # feature names except the first column (molecule ID)

In [9]:
# find five false positive molecules
drop_idx = molecule_matrix[(molecule_matrix['outcome']=='active') & (molecule_matrix['Row'].isin(fp_id))].index
molecule_matrix.drop(drop_idx,inplace=True)
molecule_matrix = molecule_matrix.iloc[:,:181]
molecule_matrix = np.array(molecule_matrix.dropna(axis=0))

In [10]:
%%time
for file in molecule_descriptor_files[1:]:
    f = pd.read_csv(file,dtype={'Row':'str'})
    f = f.iloc[:,:181] # the first column is molecule ID column
    f = np.array(f.dropna(axis=0)) # drop rows with NA 
    molecule_matrix = np.vstack((molecule_matrix,f))

CPU times: user 9.97 s, sys: 1.94 s, total: 11.9 s
Wall time: 11.9 s


In [11]:
label = molecule_matrix[:,molecule_matrix.shape[1]-1]

In [12]:
label[label=='active']=1
label[label=='inactive']=0

In [13]:
label = label.astype(int)

In [14]:
moleculeID = molecule_matrix[:,0]

In [15]:
molecule_matrix = molecule_matrix[:,1:180]

In [16]:
print('Number of molecule ID:',len(moleculeID))
print('Number of label:',len(label))
print('Number of features:',len(features))
print('Shape:',molecule_matrix.shape)

Number of molecule ID: 307783
Number of label: 307783
Number of features: 179
Shape: (307783, 179)


In [17]:
Counter(label)

Counter({0: 304501, 1: 3282})

In [18]:
# remove attributes of no variation 
remove_col = []
for col in range(molecule_matrix.shape[1]):
    temp_sum = np.sum(molecule_matrix[:,col])
    if temp_sum == 0 or temp_sum == 307783:
        remove_col.append(col)

In [19]:
remove = np.ones(molecule_matrix.shape[1]).astype('bool')
remove[np.array(remove_col)] = False
molecule_matrix = molecule_matrix[:,remove]

In [40]:
print('After removing attributes without any variation:', molecule_matrix.shape)

After removing attributes without any variation: (307783, 158)


### Handling the imbalance

##### 1). Under-sampling: Random majority under-sampling with replacement

In [20]:
molecule_matrix = molecule_matrix.astype('float')

In [42]:
%%time
from imblearn.under_sampling import RandomUnderSampler

ratio = {0:4000,1:3200}
molecule_matrix = sparse.csr_matrix(molecule_matrix)
undersam = RandomUnderSampler(ratio=ratio,return_indices=True)
X_resample,y_resample, idx_resample = undersam.fit_sample(molecule_matrix,label)

# # under-sampling
# # make use of k-means
# # source: http://contrib.scikit-learn.org/imbalanced-learn/stable/under_sampling.html
# from imblearn.under_sampling import ClusterCentroids
# cc = ClusterCentroids(random_state=0)
# X_resample, y_resample = cc.fit_sample(xtrain,y_train)
# print(sorted(Counter(y_resample).items()))

CPU times: user 1.36 s, sys: 126 ms, total: 1.49 s
Wall time: 2.96 s


In [43]:
Counter(y_resample)

Counter({0: 4000, 1: 3200})

### Split the train and test

In [44]:
x_train,x_test,y_train,y_test = train_test_split(X_resample,y_resample)

In [45]:
# remove attributes without any variation
remove = np.where(np.sum(x_train.toarray(),axis=0)==0)[0]
ind = np.ones(x_train.shape[1]).astype('bool')
ind[remove] = False
x_train = x_train[:,ind]
x_test = x_test[:,ind]

In [64]:
# active class ratio
y_tr = dict(Counter(y_train))
print(y_tr[1]/(y_tr[0]+y_tr[1]))
y_te = dict(Counter(y_test))
print(y_te[1]/(y_te[0]+y_te[1]))

0.4414814814814815
0.4533333333333333


In [67]:
print('Baseline:', (y_tr[1]/(y_tr[0]+y_tr[1])))

Baseline: 0.4414814814814815


### To be deleted

In [21]:
x_train,x_test,y_train,y_test = train_test_split(molecule_matrix,label)

In [22]:
rf = RandomForestClassifier()
rf.fit(x_train,y_train)
pred = rf.predict(x_test)
print('Accracy using all data:', accuracy_score(y_pred=pred,y_true=y_test))

Accracy using all data: 0.988069555273


### Random Forest Classifier

In [59]:
highest_cross_val_accu = -1
best_t = None
indices = range(x_train.shape[0])
tree = np.arange(10,500,50)
kf = KFold(n_splits=5,shuffle=True,random_state=28584096)

for n_tree in tree:
    accuracies = []
    for train_indices, val_indices in kf.split(indices):
        x_tr = x_train[train_indices]
        y_tr = y_train[train_indices]
        cf = RandomForestClassifier(n_estimators=n_tree)
        cf.fit(x_tr,y_tr)
        pred = cf.predict(x_train[val_indices])
        acc = accuracy_score(y_true=y_train[val_indices],y_pred=pred)
        accuracies.append(acc)
    cross_val_acc = np.mean(accuracies)
    print('N_tree: ', n_tree, 'cross validation accuracy:', cross_val_acc)
    if cross_val_acc > highest_cross_val_accu:
        highest_cross_val_accu = cross_val_acc
        best_t = n_tree
print('Best N_tree: ',best_t, '; cross-validation accuracy: ',highest_cross_val_accu)

N_tree:  10 cross validation accuracy: 0.546851851852
N_tree:  60 cross validation accuracy: 0.552592592593
N_tree:  110 cross validation accuracy: 0.558148148148
N_tree:  160 cross validation accuracy: 0.560740740741
N_tree:  210 cross validation accuracy: 0.560555555556
N_tree:  260 cross validation accuracy: 0.559259259259
N_tree:  310 cross validation accuracy: 0.561296296296
N_tree:  360 cross validation accuracy: 0.560925925926
N_tree:  410 cross validation accuracy: 0.556481481481
N_tree:  460 cross validation accuracy: 0.559259259259
Best N_tree:  310 ; cross-validation accuracy:  0.561296296296


In [77]:
rf_cv = RandomForestClassifier(n_estimators=best_t)
rf_cv.fit(x_train,y_train)
# pred_cv_train = rf_cv.predict(x_train)
y_pred = rf_cv.predict(x_test)
# print('Train accuracy:', accuracy_score(y_true=y_train,y_pred=pred_cv_train))
print('Test accuracy:', accuracy_score(y_true=y_test,y_pred=y_pred))

Test accuracy: 0.545


In [68]:
tn,fp,fn,tp = confusion_matrix(y_pred=y_pred,y_true=y_test).ravel()

In [69]:
confusion_matrix(y_pred=y_pred,y_true=y_test)

array([[737, 247],
       [548, 268]])

In [70]:
print('Recall:', tp/(tp+fn))
print('Precision:', tp/(tp+fp))

Recall: 0.328431372549
Precision: 0.520388349515


In [79]:
print('False Positive Rate:',fp/(fp+tn))
# print('True Positive Rate:',tp/(tp+fn))
print('False Negative Rate:',fn/(tp+fn))

False Positive Rate: 0.251016260163
False Negative Rate: 0.671568627451


In [76]:
features[np.argsort(rf_cv.feature_importances_)[::-1][:5]]

Index(['HYP_07_HYP', 'WBN_GC_L_0.75', 'ARC_07_HYP', 'ARC_01_ARC',
       'ARC_03_ARC'],
      dtype='object')

### Gradient Boosting

In [83]:
highest_cross_val_accu = -1
best_d = None
indices = range(x_train.shape[0])
depth = np.arange(2,15,1)
kf = KFold(n_splits=5,shuffle=True,random_state=28584096)

for d in depth:
    accuracies = []
    for train_indices, val_indices in kf.split(indices):
        x_tr = x_train[train_indices]
        y_tr = y_train[train_indices]
        cf = GradientBoostingClassifier(max_depth=d, learning_rate=0.1, subsample=0.5,random_state=8584096)
        cf.fit(x_tr,y_tr)
        pred = cf.predict(x_train[val_indices])
        acc = accuracy_score(y_true=y_train[val_indices],y_pred=pred)
        accuracies.append(acc)
    cross_val_acc = np.mean(accuracies)
    print('Depth of tree: ', d, 'cross validation accuracy:', cross_val_acc)
    if cross_val_acc > highest_cross_val_accu:
        highest_cross_val_accu = cross_val_acc
        best_d = d
print('Best depth of tree: ',best_d, '; cross-validation accuracy: ',highest_cross_val_accu)

Depth of tree:  2 cross validation accuracy: 0.544444444444
Depth of tree:  3 cross validation accuracy: 0.547037037037
Depth of tree:  4 cross validation accuracy: 0.543148148148
Depth of tree:  5 cross validation accuracy: 0.550555555556
Depth of tree:  6 cross validation accuracy: 0.543703703704
Depth of tree:  7 cross validation accuracy: 0.537222222222
Depth of tree:  8 cross validation accuracy: 0.53462962963
Depth of tree:  9 cross validation accuracy: 0.548333333333
Depth of tree:  10 cross validation accuracy: 0.539444444444
Depth of tree:  11 cross validation accuracy: 0.542592592593
Depth of tree:  12 cross validation accuracy: 0.537222222222
Depth of tree:  13 cross validation accuracy: 0.538518518519
Depth of tree:  14 cross validation accuracy: 0.537592592593
Best depth of tree:  5 ; cross-validation accuracy:  0.550555555556


In [84]:
gb = GradientBoostingClassifier(max_depth=best_d, learning_rate=0.1, subsample=0.5,random_state=8584096)
gb.fit(x_train,y_train)
y_pred = gb.pred(x_test)
print('Test accuracy:', accuracy_score(y_true=y_test,y_pred=y_pred))

AttributeError: 'GradientBoostingClassifier' object has no attribute 'pred'

### SVM

In [ ]:
highest_cross_val_accu = -1
best_c = None
indices = range(x_train.shape[0])
c = [1e-7,1e-5,1e-3,1,10,100]
kf = KFold(n_splits=5, shuffle=True, random_state=28584096)

for c_ in c:
    accuracies = []
    for train_indices, val_indices in kf.split(indices):
        x_tr = x_train[train_indices]
        y_tr = y_train[train_indices]
        svm = SVC(kernel='linear',C=c_)
        svm.fit(x_tr,y_tr)
        pred = svm.predict(x_train[val_indices])
        acc = accuracy_score(y_pred=pred,y_true=y_train[val_indices])
        accuracies.append(acc)
    cross_val_acc = np.mean(accuracies)
    print('C:', c_, ' cross validation accuracy:', cross_val_acc)
    if cross_val_acc > highest_cross_val_accu:
        highest_cross_val_accu = cross_val_acc
        best_c = c_
print('Best c:', best_c, '; cross-validation accuracy:',highest_cross_val_accu)

C: 1e-07  cross validation accuracy: 0.551666666667
C: 1e-05  cross validation accuracy: 0.551666666667
C: 0.001  cross validation accuracy: 0.551666666667


### RF:  Drug Bank

In [47]:
drugbank = pd.read_csv('DrugBank_MV.csv',dtype={'Row':'str'})
drugid = np.array(drugbank.iloc[:,0].dropna(axis=0))
drugbank = drugbank.iloc[:,1:180]
drugbank = np.array(drugbank.dropna(axis=0))
print(drugbank.shape)

(8722, 179)


In [48]:
drug_pred = rf_cv.predict(drugbank)
print(Counter(drug_pred))
# predicted active molucule
drugid_rf = drugid[np.where(drug_pred==1)]

Counter({0: 8104, 1: 618})


### SVM:  Drug Bank

In [49]:
drug_pred_svm = svm.predict(drugbank)
print(Counter(drug_pred_svm))
drugid_svm = drugid[np.where(drug_pred_svm==1)]

Counter({0: 6264, 1: 2458})


In [57]:
drug_name_rf = [drug_map[idx] for idx in drugid_rf]

In [58]:
drug_name_svm = [drug_map[idx] for idx in drugid_svm]

In [59]:
name = [set(drug_name_svm) & set(drug_name_rf)]